## Problem 1 solution

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

soup = BeautifulSoup(source, 'html')

table=soup.find('table')

# Create our own dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for tr_cell in table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip()) # 去除首尾空格
    if len(row_data)==3:
        df.loc[len(df)] = row_data # assign the first row data to the first line of df

        
df = df[df['Borough'] != 'Not assigned']

temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)) # join two Neighborhoods
# %s 字符串
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

df_merge = pd.merge(df, temp_df, on='Postalcode')
df_merge.drop(['Neighborhood'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns = {'Neighborhood_joined': 'Neighborhood'}, inplace=True)

## Problem 2

In [12]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [13]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [36]:
# geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, left_on='Postal Code', right_on='Postalcode')
geo_merged.head(2)

,Postal Code,Latitude,Longitude,Postalcode,Borough,Neighborhood
0,M1B,43.806686,-79.194353,M1B,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


In [37]:
geo = geo_merged.drop(['Postal Code'], axis=1)

In [38]:
geo.head()

,Latitude,Longitude,Postalcode,Borough,Neighborhood
0,43.806686,-79.194353,M1B,Scarborough,"Rouge, Malvern"
1,43.784535,-79.160497,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,43.763573,-79.188711,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,43.770992,-79.216917,M1G,Scarborough,Woburn
4,43.773136,-79.239476,M1H,Scarborough,Cedarbrae


In [39]:
fixed_columns = list(geo.columns[2:5]) + list(geo.columns[0:2])
geo = geo[fixed_columns]

In [40]:
geo.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
